<a href="https://colab.research.google.com/github/wallisonferreira/machine-learning-pavic/blob/main/PAVIC_ML_04_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

A melhor maneira de pensar em redes neurais é como circuitos de valores reais. Mas, ao invés de valores booleanos, valores reais e, ao invés de portas lógicas como **and** ou **or**, portas binárias (dois operandos) como $*$ (multiplicação), + (adição), max, exp, etc. Além disso, também teremos **gradientes** fluindo pelo circuito, mas na direção oposta.

In [15]:
#Slide 43-46
#multiply gate
def forwardMultiplyGate(a, b):
    return a * b

De forma matemática, a gente pode considerar que essa porta implementa a seguinte função:

$$f(x,y)=x*y$$

## O Objetivo

Vamos imaginar que temos o seguinte problema:
1. Nós vamos providenciar a um circuito valores específicos como entrada (x=1, y=-3)
2. O circuito vai calcular o valor de saída (-3)
3. A questão é: *Quanto mudar a entrada para levemente **aumentar** a saída?*

No nosso caso, em que direção devemos mudar x,y para conseguir um número maior que -3? Note que, pro nosso exemplo, se x = 0.99 e y = -2.99, x$*$y = -2.96 que é maior que -3. **-2.96 é melhor (maior) que -3**, e obtivemos uma melhora de 0.04.

## Estratégia 1: Busca Aleatória

Ok. Isso não é trivial? A gente pode simplesmente gerar valores aleatórios, calcular a saída e guardar o melhor resultado.

In [16]:
x, y = 1, -3

# calcula a multriplicação das entradas
melhor_saida = forwardMultiplyGate(x,y)

# tigela para receber a melhor saída para x e y
melhor_x, melhor_y = 0, 0


for k in range(0,100):
    x_try = 5*np.random.random() - 5
    y_try = 5*np.random.random() - 5
    out = forwardMultiplyGate(x_try, y_try)

    if out > melhor_saida:
        melhor_saida = out
        melhor_x, melhor_y = x_try, y_try

print(melhor_x, melhor_y, forwardMultiplyGate(melhor_x, melhor_y))

-4.778148712380367 -4.495715453819305 21.481197006895233


Ok, foi bem melhor. Mas, e se tivermos milhões de entradas? É claro que essa estratégia não funcionará. Vamos tentar algo mais aprimorado.

## Estratégia 2: Busca Aleatória Local

Um passo aleatorio em qualquer direçao, torcer para tomar a decisao correta
agora temos um passo pra evitar um pulo muito longo


In [17]:
x, y = 1, -3
passo = 0.01
melhor_saida = forwardMultiplyGate(x,y)
melhor_x, melhor_y = 0, 0

for k in range(0,100):
    x_try = x + passo * (2*np.random.random() - 1)
    y_try = y + passo * (2*np.random.random() - 1)
    out = forwardMultiplyGate(x_try, y_try)

    if out > melhor_saida:
        melhor_saida = out
        melhor_x, melhor_y = x_try, y_try

print(melhor_x, melhor_y, forwardMultiplyGate(melhor_x, melhor_y))

0.9903999077053768 -2.99025353255345 -2.9615468226566137


Otimoooo! Demos um passinho mais controlado -2.96 é menor que -3.... mas precisamos de algo melhor, uma estratégia mais inteligente.

## Estratégia 3: Gradiente Numérico

Imagine agora que a gente pega as entradas de um circuito e puxa-as para uma direção positiva. Essa força puxando $x$ e $y$ vai nos dizer como $x$ e $y$ devem mudar para aumentar a saída. Não entendeu? Vamos explicar:

Se olharmos para as entradas, a gente pode intuitivamente ver que a força em $y$ deveria sempre ser positiva, porque tornando $y$ um pouquinho maior de $y=1$ para $y=-1$ aumenta a saída do circuito para $-1$, o que é bem maior que $-3$. Por outro lado, se a força em $x$ for negativa, tornando-o menor, como de $x=1$ para $x=0.5$, também aumenta a saída: $-0.5\times-3 = -1.5$, de novo maior que $-3$.

E como calcular essa força? Usando **derivadas**.

> *A derivada pode ser pensada como a força que a gente aplica em cada entrada para aumentar a saída*


E como exatamente a gente vai fazer isso? Em vez de olhar para o valor de saída, como fizemos anteriormente, nós vamos iterar sobre as cada entrada individualmente, aumentando-as bem devagar e vendo o que acontece com a saída. **A quantidade que a saída muda é a resposta da derivada**.

Vamos para definição matemática. A derivada em relação a $x$ pode ser definida como:

$$\frac{\partial f(x,y)}{\partial x} = \frac{f(x+h,y) - f(x,y)}{h}$$

Onde $h$ é pequeno. Nós vamos, então, calcular a saída inicial $f(x,y)$ e aumentar $x$ por um valor pequeno $h$ e calcular a nova saída $f(x+h,y)$. Então, nós subtraimos esse valores para ver a diferença e dividimos por $f(x+h,y)$ para normalizar essa mudança pelo valor (arbitrário) que nós usamos.

Em termos de código, teremos:

In [18]:
#Slide 49-52
x, y = 1, -3
out = forwardMultiplyGate(x,y) # f(x,y)
h=0.01 #h -> 0

# derivada em relação a x
out2 = forwardMultiplyGate(x+h, y) # f(x+h, y)

derivada_x = (out2 - out) / h

# derivada em relação a y
out3 = fowardMultiplyGate(x, y+h)

derivada_y = (out3 - out ) / h

# resultados
derivada_x, derivada_y # vetor de gradiente

(-3.000000000000025, 0.9999999999999787)

Essas saídas representam a foça da atualização naquele ponto
o *3* indica que a força da derivada de $x$ é maior que a força da derivada de $y$
O sinal indica somente se a função cresce ou decresce

> *A derivada em relação a alguma entrada pode ser calculada ajustando levemente aquela entrada e observando a mudança no valor da saída*

A derivada é calculada sobre cada entrada, enquanto o **gradiente** representa todas as derivadas sobre as entradas concatenadas em um vetor.
> Derivada é so um valor e o gradiente eh o vertor desses valores

In [19]:
#Slide 49-52
x, y = 1, -3
passo = 0.001
out = forwardMultiplyGate(x, y) # f(x , y)

x = x + passo * derivada_x
y = x + passo * derivada_y

out_new = forwardMultiplyGate(x, y)

print(out, out_new)

-3 0.995006


Dessa forma conseguimos reduzir nossa função em poucas iterações (rodar mais 2x)

**Passo maior nem sempre é melhor**: É importante destacar que qualquer valor de passo maior que 0.01 ia sempre funcionar melhor (por exemplo, passo = 1 gera a saída = 1). No entanto, à medida que os circuitos vão ficando mais complexos (como em redes neurais completas), a função vai ser tornando mais caótica e complexa. O gradiente garante que se você tem um passo muito pequeno (o ideal seria infinitesimal), então você definitivamente aumenta a saída seguindo aquela direção. O passo que estamos utilizando (0.01) ainda é muito grande, mas como nosso circuito é simples, podemos esperar pelo melhor resultado. Lembre-se da analogia do **escalador cego**.

## Estratégia 4: Gradiente Analítico

A estratégia que utilizamos até agora de ajustar levemente a entrada e ver o que acontece com a saída pode não ser muito cômoda na prática quando temos milhares de entradas para ajustar. Então, a gente precisa de algo melhor.

Felizmente, existe uma estratégia mais fácil e muito mais rápida para calcular o gradiente: podemos usar cálculo para derivar diretamente a nossa função. Chamamos isso de **gradiente analítico** e dessa forma não precisamos ajustar levemente nada.

> *O gradiente analítico evita o leve ajustamento das entradas. O circuito pode ser derivado usando cálculo.*

É muito fácil calcular derivadas parciais para funções simples como $x*y$. Se você não lembra da definição, aqui está o cálculo da derivada parcial em relação a $x$ da nossa função $f(x,y)$:

$$\frac{\partial f(x,y)}{\partial x} = \frac{f(x+h,y) - f(x,y)}{h}
= \frac{(x+h)y - xy}{h}
= \frac{xy + hy - xy}{h}
= \frac{hy}{h}
= y$$

A derivada parcial em relação em $x$ da nossa $f(x,y)$ é igual $y$. Você reparou na coincidência de $\partial x = 3.0$, que é exatamente o valor de $y$? E que o mesmo aconteceu para $x$? **Então, a gente não precisa ajustar nada!** E nosso código fica assim:

In [21]:
#Slide 49-52
x, y = 1, -3
out = forwardMultiplyGate(x,y)

derivada_x = y
derivada_y = x

passo = 0.001

x = x + passo * derivada_x
y = y + passo * derivada_y

out_new = forwardMultiplyGate(x,y)
print(out, out_new)

-3 -2.990003


É importante destacar que a Estratégia #3 reduziu a #2 para uma única vez. Porém, a #3 nos dá somente uma aproximação do gradiente, enquanto a Estratégia #4 nos dá o valor exato. Sem aproximações. O único lado negativo é que temos de saber derivar a nossa funcão.

Recapitulando o que vimos até aqui:
- __Estratégia 1__: definimos valores aleatórios em todas as iterações. Não funciona para muitas entradas.
- __Estratégia 2__: pequenos ajustes aleatórios nas entradas e vemos qual funciona melhor. Tão ruim quando a #1.
- __Estratégia 3__: muito melhor através do cálculo do gradiente. Independentemente de quão complicado é o circuito, o **gradiente numérico** é muito simples de se calcular (mas um pouco caro).
- __Estratégia 4__: no final, vimos que a forma melhor, mais inteligente e mais rápida é calcular o **gradiente analítico**. O resultado é idêntico ao gradiente numérico, porém mais rápido e não precisa de ajustes.

## Caso Recursivo: Múltiplas Portas

Como vamos calcular agora a nossa derivada? Primeiramente, vamos esquecer da porta de soma e fingir que temos apenas duas entradas no nosso circuito: **q** e **z**. Como já vimos, as nossas derivadas parciais podem ser definidas da seguinte maneira:

$$f(q,z) = q z \hspace{0.5in} \implies \hspace{0.5in} \frac{\partial f(q,z)}{\partial q} = z, \hspace{1in} \frac{\partial f(q,z)}{\partial z} = q$$

Ok, mas e em relação a $x$ e $y$? Como $q$ é calculado em função de $x$ e $y$ (pela adição em nosso exemplo), nós também podemos calcular suas derivadas parciais:

$$q(x,y) = x + y \hspace{0.5in} \implies \hspace{0.5in} \frac{\partial q(x,y)}{\partial x} = 1, \hspace{1in} \frac{\partial q(x,y)}{\partial y} = 1$$

Correto! As derivadas parciais são 1, independentemente dos valores de $x$ e $y$. Isso faz sentido se pensarmos que para aumentar A saída de uma porta de adição, a gente espera uma força positiva tanto em $x$ quanto em $y$, independente dos seus valores.

Com as fórmulas acima, nós sabemos calcular o gradiente da saída em relação a $q$ e $z$, e o gradiente de $q$ em relação a $x$ e $y$. Para calcular o gradiente do nosso circuito em relação a $x$, $y$ e $z$, nós vamos utilizar a **Regra da Cadeia**, que vai nos dizer como combinar esses gradientes. A derivada final em relação a $x$, será dada por:

$$\frac{\partial f(q,z)}{\partial x} = \frac{\partial q(x,y)}{\partial x} \frac{\partial f(q,z)}{\partial q}$$

Pode parecer complicado à primeira vista, mas a verdade é que isso vai ser simplificado a somente duas multiplicações:

In [22]:
#Slide 53-56
def forwardAddGate(a, b):
    return a + b

def forwardCircuit(x, y, z):
    q = forwardAddGate(x, y)
    f = forwardMultiplyGate(q, z)
    return f

print(forwardCircuit(-2, 5, 4))

12


In [23]:
#Slide 53-56

x, y, z = -2, 5, -4

q = forwardAddGate(x, y)
f = forwardMultiplyGate(q, z)

# derivada da porta de multiplicação
der_f_z = q
der_f_q = z

# derivada da porta de adição
der_q_x = 1
der_q_y = 1

# regra da cadeia (função interna * função externa)
der_f_x = der_f_q * der_q_x
der_f_y = der_f_q * der_q_y
der_f_z = der_f_z

print(der_f_x, der_f_y, der_f_z)

-4 -4 3


In [24]:
# Atualizando os pesos
passo = 0.001

x = x + passo * der_f_x
y = y + passo * der_f_y
z = z + passo * der_f_z
x, y, z

(-2.004, 4.996, -3.997)

O sinal negativo nos indica se devemos crescer ou decrescer aquela entrada
e o valor indica com que força

In [25]:
# Vetor gradiente
grad_f_rel_xyz = [der_f_x, der_f_y, der_f_z]

print(forwardCircuit(x, y, z)) # > -12

-11.959024000000001


Agora atualizaremos os nossos pesos com o gradiente

In [ ]:
#Slide 53-56

## Checagem do gradiente numérico

Vamos verificar se os gradientes analíticos que calculamos por backpropagation estão corretos. Lembre-se que podemos fazer isso através do gradiente numérico e esperamos que o resultado seja [-4, -4, 4] para $x,y,z$.

In [32]:
#Slide 57
x, y, z = -2, 5, -4
h = 0.001

der_x = (forwardCircuit(x + h, y, z) - forwardCircuit(x, y, z)) / h
der_y = (forwardCircuit(x, y + h, z) - forwardCircuit(x, y, z)) / h
der_z = (forwardCircuit(x, y, z + h) - forwardCircuit(x, y, z)) / h

print(der_x, der_y, der_z)

-3.9999999999995595 -4.000000000001336 3.0000000000001137


## Neurônio Sigmóide

Qualquer função diferenciável pode atuar como uma porta, como também podemos agrupar múltiplas portas para formar uma simples porta, ou decompor um função em múltiplas portas quando for conveniente. Para exemplificar, vamos utilizar a função de ativação *sigmoid* com entradas **x** e pesos **w**:
> Neste caso o bias é representado pelo w2

$$f(w,x) = \frac{1}{1+e^{-(w_0x_0 + w_1x_1 + w_2)}}$$

Como dito, a função acima nada mais é que a função sigmoid $\sigma(x)$. Sabendo, então, que a derivada da função sigmoid é:

$$\sigma(x)=\frac{1}{1+e^{-x}}=(1-\sigma(x))\sigma(x)$$

Vamos calcular a gradiente em relação as entradas:

![This is an image](https://github.com/mafaldasalomao/pavic_treinamento_ml/blob/main/Machine_Learning/figures/sigmoid_example_backprop.png?raw=true)

In [ ]:
#forward do sigmoid
# vamos tomar o w0 e w1 como entradas
# O w2 será considerado como bias
#Slide 62
#Primeiro vamos calcular a derivada do neuronio sigmoid com as derivadas conhecidas!!  (se soubermos as derivadas) gradiente analítico
#Depois calcularemos a derivada do neuronio sigmoid com circuitos... quebradinho (se NAO soubermos as derivadas) gradiente numérico

w0, w1, w2 = 2, -3, -3
x0, x1 = -1, -2



In [ ]:
#Vamos ver primeiro as questoes do backpropagation antes de continuarmos a implementação do sigmoid
#Slide 63-74